In [1]:
%pwd
%cd ../..

/home/rydevera3/data-science/umn_research/Deeplifting


In [2]:
import glob

import numpy as np
import pandas as pd

from deeplifting.problems import PROBLEMS_BY_NAME

In [58]:
# Get the results from deeplifting
files = glob.glob('./results/results-2023-08-03-2*')
deeplifting_results = pd.read_parquet(files)

# Select specific columns for results
deeplifting_results = (
    deeplifting_results[['problem_name', 'algorithm', 'x1', 'x2', 'f', 'total_time', 'global_minimum']]
)

# For now let's remove sine-envelope function because there is something
# going on there
mask = deeplifting_results['problem_name'] != 'sine_envelope'
deeplifting_results = deeplifting_results.loc[mask].reset_index(drop=True)
deeplifting_results['hits'] = np.abs(
    deeplifting_results['global_minimum'] - deeplifting_results['f']
) <= 1e-4
deeplifting_results['hits'] = deeplifting_results['hits'].astype(int)
deeplifting_results = deeplifting_results.rename(columns={'total_time': 'time'})
deeplifting_results = deeplifting_results.drop(columns=['global_minimum'])
deeplifting_results.head()

,problem_name,algorithm,x1,x2,f,time,hits
0,kriging_peaks_red010,Deeplifting,-2.169612,0.295735,0.291121,440.880969,1
1,kriging_peaks_red010,Deeplifting,-2.169612,0.295735,0.291121,3123.716541,1
2,kriging_peaks_red010,Deeplifting,-2.169612,0.295735,0.291121,112.328760,1
3,kriging_peaks_red010,Deeplifting,-2.169612,0.295735,0.291121,254.529593,1
4,kriging_peaks_red010,Deeplifting,-2.169612,0.295735,0.291121,2216.122304,1


In [60]:
# Load in the comparisons
comparison_files = glob.glob('./results/algorithm-comparisons-*.parquet')
comparison_results = pd.read_parquet(comparison_files)

# # Omit sine envelope function
# mask = comparison_results['problem_name'] != 'sine_envelope'
# comparison_results = comparison_results.loc[mask].reset_index(drop=True)

comparison_results.loc[comparison_results['problem_name'] == 'ackley']

,x1,x2,f,algorithm,time,problem_name,hits
1440,-1.998055,10.989293,15.882023,IPOPT,0.018117,ackley,0
1441,-4.997831,13.993926,17.555579,IPOPT,0.012923,ackley,0
1442,8.997289,-13.995782,18.099026,IPOPT,0.019353,ackley,0
1443,2.998540,13.993183,17.358415,IPOPT,0.018837,ackley,0
1444,30.999359,-2.999938,19.755555,IPOPT,0.012196,ackley,0
...,...,...,...,...,...,...,...
1515,9.702870,0.342164,17.318820,PyGRANSO,0.229584,ackley,0
1516,-16.624425,-4.712389,20.919912,PyGRANSO,0.202239,ackley,0
1517,-25.442471,-6.222187,15.267776,PyGRANSO,0.345149,ackley,0
1518,-430.056199,524.584960,15.882023,PyGRANSO,0.279412,ackley,0


In [61]:
comparison_results['problem_name'].value_counts()

problem_name
ackley3                 80
levy                    80
griewank                80
schaffer_n4             80
mathopt6                80
eggholder               80
bukin_n6                80
ex8_1_1                 80
holder_table            80
shubert                 80
schwefel                80
kriging_peaks_red030    80
cross_in_tray           80
drop_wave               80
schaffer_n2             80
quantum                 80
kriging_peaks_red020    80
rosenbrock              80
ackley                  80
sine_envelope           80
kriging_peaks_red010    80
cross_leg_table         80
rastrigin               80
levy_n13                80
ackley2                 80
Name: count, dtype: int64

In [62]:
results_df = pd.concat((comparison_results, deeplifting_results))

In [63]:
aggregated_results_df = (
    results_df
    .groupby(['problem_name', 'algorithm'])
    .agg({'hits': 'mean'})
    .rename(columns={'hits': 'Success Rate'})
    .unstack()
    .transpose()
    .reorder_levels(['algorithm', None])
    .transpose()
)

In [64]:
aggregated_results_df.mean()

algorithm                           
Deeplifting             Success Rate    0.840476
Differential Evolution  Success Rate    0.688000
Dual Annealing          Success Rate    0.724000
IPOPT                   Success Rate    0.246000
PyGRANSO                Success Rate    0.232000
dtype: float64

In [65]:
aggregated_results_df

algorithm,Deeplifting,Differential Evolution,Dual Annealing,IPOPT,PyGRANSO
,Success Rate,Success Rate,Success Rate,Success Rate,Success Rate
problem_name,,,,,
ackley,1.00,1.00,1.00,0.05,0.00
ackley2,NaN,0.90,1.00,1.00,0.05
ackley3,NaN,0.00,0.00,0.00,0.00
bukin_n6,0.35,0.00,0.00,0.00,0.00
cross_in_tray,1.00,1.00,1.00,0.05,0.30
cross_leg_table,0.65,0.05,0.00,0.00,0.05
drop_wave,1.00,0.55,0.70,0.10,0.00
eggholder,0.45,0.20,0.45,0.00,0.05


In [54]:
aggregated_results_df.mean().sort_values(ascending=False)

algorithm                           
Deeplifting             Success Rate    0.840476
Dual Annealing          Success Rate    0.814286
Differential Evolution  Success Rate    0.776190
PyGRANSO                Success Rate    0.273810
IPOPT                   Success Rate    0.245238
dtype: float64

In [55]:
aggregated_results_df.shape

(21, 5)